In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
#loading the data
data = pd.read_csv('/home/guilherme/Documentos/repos/ds_ao_dev/projeto_webscraping/df_raw.csv')

#delete the columns not necessery
data = data.drop(['Unnamed: 0' ],axis=1)

#product id

data = data.dropna(subset=['product_id'])

#product category

#product name
data['products_name']= data['products_name'].apply(lambda x: x.replace(' ','_').lower())
#products_price

#já fiz o tratamento anteriomente

#product datetime


data['scrapy_datetime']= pd.to_datetime(data['scrapy_datetime'])

#style id

#color id

#color name
data['color_name']= data['color_name'].apply(lambda x: x.replace(' ','_').replace('/','_').lower() if pd.notnull(x) else x)

#Composition

data['Composition']= data['Composition'].where(~data['Composition'].str.contains('Pocket lining:',na=False) ) 

data['Composition']= data['Composition'].where(lambda x: ~x.str.contains('Shell:',na=False) )

data['Composition'] = data['Composition'].where(lambda x: ~x.str.contains('Lining:',na=False) )

#drop duplicates
#first were making drop the that are duplicates (excect the colum Composition), because the colums is duplicate
data= data.drop_duplicates(subset= ['product_id', 'products_category', 'products_name', 'products_price',
       'scrapy_datetime', 'stlye_id', 'color_id', 'color_name'],keep='last')

   
#reset index
data = data.reset_index(drop=True)


#break composition by comma (separando as composições dos materiais por coluna)
df1 = data['Composition'].str.split(',',expand=True)


#Cotton| Spandex | Polyester |  Elastomultiester

data.head()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,dark_denim_blue,NaN
1,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,black,NaN
2,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,denim_blue,NaN
4,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,dark_blue,NaN


In [3]:
#Cotton| Spandex | Polyester |  Elastomultiester
#2 etapa
#definir as colunas (organizando as colunas por máteria-prima)

#criando Dataframe vazio com o mesmo tamanho do dataframe original para prencher com as respectivas composições
df_ref = pd.DataFrame(index=np.arange(len(data) ),columns=['cotton','spandex','polyester','elastomultiester'])

#cotton
#pegando a coluna da composição cotton
df_cotton = df1[0]

#colocando o nome na coluna 
df_cotton.name = 'cotton'
#mesclando com a tabela vazia 
df_ref = pd.concat([df_ref,df_cotton],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['cotton'] = df_ref['cotton'].fillna( 'Cotton 0%' )

#polyester 

#adicionando as coluna da composição Polyester
df_polyester =df1[1].where(lambda x: x.str.contains('Polyester',na=True))

#renomeando a coluna
df_polyester.name= 'polyester'


df_ref= pd.concat([df_ref,df_polyester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['polyester'] = df_ref['polyester'].fillna( 'Polyester 0%' )

#Elastomultiester

#coletando apenas a composição elastano
df_elastomultiester =df1[1].where(lambda x: x.str.contains('Elastomultiester',na=True))

#renomenado a coluna eslastano
df_elastomultiester.name= 'elastomultiester'


#mesclando com df_ref vazio e com a mesma quantidade de linhas que há no index
df_ref= pd.concat([df_ref,df_elastomultiester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['elastomultiester'] = df_ref['elastomultiester'].fillna( 'Elastomultiester 0%' )

#Spandex

#coletando composição spandex da segunda coluna df1

df_spandex= df1[1].where(lambda x: x.str.contains('Spandex',na=True))

df_spandex.name = 'spandex'

#combine from spandex botth columns 1 and 2

df_spandex = df_spandex.combine_first(df1[2])


df_ref= pd.concat([df_ref,df_spandex],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['spandex'] = df_ref['spandex'].fillna('Spandex 0%')

#coletando composição spandex da terceira coluna do df1

#df_spandex_2= df1[2].where(lambda x: x.str.contains('Spandex',na=True))
             
#df_spandex_2.name= 'spandex'

#df_ref= pd.concat([df_ref,df_spandex_2],axis=1)

#df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]



#final join

data= pd.concat([data,df_ref],axis=1)

#coletando apenas os numeros e transformando em números inteiros

#format composition data

#cotton
data['cotton']= data['cotton'].apply(lambda x: int(re.search('\d+',x ).group(0))/100 if pd.notnull(x) else x)

#spandex
data['spandex'] =data['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#polyester
data['polyester']= data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#elastomultiester
data['elastomultiester']=data['elastomultiester'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#drop columns
data= data.drop(['Composition'],axis=1)

#drop duplicates


data= data.drop_duplicates()

In [17]:
data.to_csv('products_hm_cleaned.csv')

In [4]:
data.sample(5)

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,cotton,polyester,elastomultiester,spandex
515,974597002,men_jeans_slim,slim_tapered_jeans,29.99,2022-04-24 11:00:00-21:30,974597,2,black,0.00,0.00,0.0,0.00
490,938875014,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,14,dark_gray,0.00,0.00,0.0,0.00
213,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,dark_blue,0.00,0.00,0.0,0.00
375,427159006,men_jeans_ripped,trashed_skinny_jeans,39.99,2022-04-24 11:00:00-21:30,427159,6,denim_blue_bleached,0.93,0.06,0.0,0.01
4,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,dark_blue,0.00,0.00,0.0,0.00
